# Getting Started with Prompt Engineering
by Kazhian Muthusami


This notebook contains examples and exercises to learning about prompt engineering.

We will be using the [OpenAI APIs](https://platform.openai.com/) for all examples. I am using the default settings `temperature=0.7` and `top-p=1`

---

## 1. Prompt Engineering Basics

Objectives
- Load the libraries
- Review the format
- Cover basic prompts
- Review common use cases

Below we are loading the necessary libraries, utilities, and configurations.

In [1]:
# %%capture
# update or install the necessary libraries
!pip install openai==1.55.3 tiktoken==0.6 session-info --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 4.0 MB/s eta 0:00:00


In [21]:
# Import all Python packages required to access the Azure Open AI API
import json
import tiktoken
import session_info
import IPython

from IPython import display
from openai import AzureOpenAI

### Create a json file using below format. Name: `openai-config.json`

```
{
    "AZURE_OPENAI_KEY": "#*9jO#**#Swg8h#*zut*#*mvJv#*99AACYeB*#*OGJa3*",
    "AZURE_OPENAI_ENDPOINT": "https://#bcd1&3*45#.openai.azure.com/",
    "AZURE_OPENAI_APIVERSION": "2024-02-15-preview:",
    "CHATGPT_MODEL": "gpt-4o-mini"
}

In [10]:
# Azure Open AI credentials and the id of the deployed chat model are stored as
# key value pairs in a json file

with open('openai-config.json', 'r') as az_creds:
    data = az_creds.read()

In [11]:
creds = json.loads(data)

# Credentials to authenticate to the personalized Open AI model server

client = AzureOpenAI(
    azure_endpoint=creds["AZURE_OPENAI_ENDPOINT"],
    api_key=creds["AZURE_OPENAI_KEY"],
    api_version=creds["AZURE_OPENAI_APIVERSION"]
)

In [12]:
# Deployment namre of the ChatCompletion endpoint
CHATGPT_MODEL = creds["CHATGPT_MODEL"]
print(f"The model I am going to use: {creds['CHATGPT_MODEL']}")

The model I am going to use: gpt-4o-mini-kazhian


In [13]:
# Utility methods

def set_open_params(
    model=creds["CHATGPT_MODEL"],
    temperature=0.7,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
):
    """ set openai parameters"""

    openai_params = {}

    openai_params['model'] = model
    openai_params['temperature'] = temperature
    openai_params['max_tokens'] = max_tokens
    openai_params['top_p'] = top_p
    openai_params['frequency_penalty'] = frequency_penalty
    openai_params['presence_penalty'] = presence_penalty
    return openai_params

def get_completion(params, messages):
    """ GET completion from openai api"""

    response = client.chat.completions.create(
        model = params['model'],
        messages = messages,
        temperature = params['temperature'],
        max_tokens = params['max_tokens'],
        top_p = params['top_p'],
        frequency_penalty = params['frequency_penalty'],
        presence_penalty = params['presence_penalty'],
    )
    return response

def print_in_markdown(text):
    """ Print text in markdown format"""
    IPython.display.Markdown(text)

Basic prompt example:

In [14]:
# basic example
params = set_open_params()

prompt = "The sky is"

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
params

{'model': 'gpt-4o-mini-kazhian',
 'temperature': 0.7,
 'max_tokens': 256,
 'top_p': 1,
 'frequency_penalty': 0,
 'presence_penalty': 0}

In [22]:
response.choices[0].message.content
display.Markdown(response.choices[0].message.content)

The sky is a vast expanse above us, often filled with clouds, stars, and the sun or moon, depending on the time of day. It can change colors with the time of day, displaying vibrant hues during sunrise and sunset. The sky also plays a crucial role in weather patterns and is home to various atmospheric phenomena. What specific aspect of the sky are you interested in?

Try with different temperature to compare results:

In [23]:
params = set_open_params(temperature=0)
response = get_completion(params, messages)


In [24]:
# Print the response
# print_in_markdown(response.choices[0].message.content)
IPython.display.Markdown(response.choices[0].message.content)

The sky is a vast expanse above us, often filled with clouds, stars, and the sun or moon, depending on the time of day. It can change colors with the time of day, displaying vibrant hues during sunrise and sunset. The sky also plays a crucial role in weather patterns and is home to various atmospheric phenomena. What specific aspect of the sky are you interested in?

### 1.1 Text Summarization

In [25]:
params = set_open_params(temperature=0.7)
prompt = """Antibiotics are a type of medication used to treat bacterial infections. They work by either killing the bacteria or preventing them from reproducing, allowing the body's immune system to fight off the infection. Antibiotics are usually taken orally in the form of pills, capsules, or liquid solutions, or sometimes administered intravenously. They are not effective against viral infections, and using them inappropriately can lead to antibiotic resistance.

Explain the above in one sentence:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Antibiotics are medications that combat bacterial infections by killing bacteria or inhibiting their reproduction, but they are ineffective against viruses and can contribute to antibiotic resistance if misused.

Exercise: Instruct the model to explain the paragraph in one sentence like "I am 5". Do you see any differences?

### 1.2 Question Answering

In [26]:
prompt = """Answer the question based on the context below. Keep the answer short and concise. Respond "Unsure about answer" if not sure about the answer.

Context: Teplizumab traces its roots to a New Jersey drug company called Ortho Pharmaceutical. There, scientists generated an early version of the antibody, dubbed OKT3. Originally sourced from mice, the molecule was able to bind to the surface of T cells and limit their cell-killing potential. In 1986, it was approved to help prevent organ rejection after kidney transplants, making it the first therapeutic antibody allowed for human use.

Question: What was OKT3 originally sourced from?

Answer:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)


Mice.

Context obtained from here: https://www.nature.com/articles/d41586-023-00400-x

Exercise: Edit prompt and get the model to respond that it isn't sure about the answer.

### 1.3 Text Classification

In [27]:
prompt = """Classify the text into neutral, negative or positive.

Text: I think the food was okay.

Sentiment:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Neutral

Exercise: Modify the prompt to instruct the model to provide an explanation to the answer selected.

### 1.4 Role Playing

In [28]:
prompt = """The following is a conversation with an AI research assistant. The assistant tone is technical and scientific.

Human: Hello, who are you?
AI: Greeting! I am an AI research assistant. How can I help you today?
Human: Can you tell me about the creation of blackholes?
AI:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

messages = [
    {
        "role": "user",
        "content": prompt
    }

]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Certainly! Black holes are formed through several astrophysical processes, primarily involving the gravitational collapse of massive objects. The most common mechanisms are:

1. **Stellar Collapse**: When a massive star exhausts its nuclear fuel, it can no longer support itself against gravitational forces. The core collapses under its own gravity, and if the remaining mass is greater than the Tolman-Oppenheimer-Volkoff limit (approximately 2-3 solar masses), it continues to collapse, forming a black hole.

2. **Supernova Events**: During the death of a massive star, a supernova explosion can occur. If the core remnant is sufficiently massive, it will collapse into a black hole post-explosion.

3. **Merging Neutron Stars**: When two neutron stars in a binary system collide, the resulting mass can surpass the neutron star limit, leading to the formation of a black hole.

4. **Primordial Black Holes**: These hypothetical black holes may have formed shortly after the Big Bang due to density fluctuations in the early universe. Their existence is still a topic of research.

5. **Supermassive Black Holes**: Found at the centers of galaxies, these black holes can form through the merging of smaller black holes and the

Exercise: Modify the prompt to instruct the model to keep AI responses concise and short.

### 1.5 Code Generation

In [29]:
prompt = "\"\"\"\nTable departments, columns = [DepartmentId, DepartmentName]\nTable students, columns = [DepartmentId, StudentId, StudentName]\nCreate a MySQL query for all students in the Computer Science Department\n\"\"\""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)


To retrieve all students from the Computer Science Department, you can use a SQL query that joins the `departments` table with the `students` table. Here is the SQL query:

```sql
SELECT s.StudentId, s.StudentName
FROM students s
JOIN departments d ON s.DepartmentId = d.DepartmentId
WHERE d.DepartmentName = 'Computer Science';
```

This query selects the `StudentId` and `StudentName` from the `students` table where the `DepartmentId` matches the `DepartmentId` in the `departments` table, filtering specifically for the department where the name is "Computer Science".

### 1.6 Reasoning

In [30]:
prompt = """The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.

Solve by breaking the problem into steps. First, identify the odd numbers, add them, and indicate whether the result is odd or even."""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Let's break down the problem step by step.

1. **Identify the odd numbers in the group:**
   The group of numbers is: 15, 32, 5, 13, 82, 7, 1.
   The odd numbers are: 15, 5, 13, 7, 1.

2. **List the identified odd numbers:**
   - 15
   - 5
   - 13
   - 7
   - 1

3. **Add the identified odd numbers:**
   Let's perform the addition step by step:
   - First, add 15 + 5 = 20
   - Next, add 20 + 13 = 33
   - Then, add 33 + 7 = 40
   - Finally, add 40 + 1 = 41

   So, the sum of the odd numbers is:
   \[
   15 + 5 + 13 + 7 + 1 = 41
   \]

4. **Determine whether the result is odd or even:**
   The sum, 41, is an odd number.

5. **Conclusion:**
   The odd numbers in the group add up

Exercise: Improve the prompt to have a better structure and output format.

## 2. Advanced Prompting Techniques

Objectives:

- Cover more advanced techniques for prompting: few-shot, chain-of-thoughts,...

### 2.2 Few-shot prompts

In [ ]:
prompt = """The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: The answer is False.

The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.
A: The answer is True.

The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.
A: The answer is True.

The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.
A: The answer is False.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

To determine if the odd numbers in the group add up to an even number, let's first identify the odd numbers in the set: 15, 5, 13, 7, and 1.

Now, let's add them up:

15 + 5 = 20  
20 + 13 = 33  
33 + 7 = 40  
40 + 1 = 41  

The sum of the odd numbers is 41, which is an odd number.

Therefore, the answer is **False**.

### 2.3 Chain-of-Thought (CoT) Prompting

In [ ]:
prompt = """The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: Adding all the odd numbers (9, 15, 1) gives 25. The answer is False.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Adding all the odd numbers (15, 5, 13, 7, 1) gives 41. The answer is False.

### 2.4 Zero-shot CoT

In [ ]:
prompt = """I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1. How many apples did I remain with?

Let's think step by step."""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Step 1: Bought 10 apples.
Step 2: Gave 2 apples to the neighbor and 2 apples to the repairman.
Remaining apples: 10 - 2 - 2 = 6 apples.
Step 3: Bought 5 more apples.
Total apples now: 6 + 5 = 11 apples.
Step 4: Ate 1 apple.
Remaining apples: 11 - 1 = 10 apples.

Final answer: You remained with 10 apples.

### 2.5 Self-Consistency
As an exercise, check examples in our [guide](https://github.com/dair-ai/Prompt-Engineering-Guide/blob/main/guides/prompts-advanced-usage.md#self-consistency) and try them here.

### 2.6 Generate Knowledge Prompting

As an exercise, check examples in our [guide](https://github.com/dair-ai/Prompt-Engineering-Guide/blob/main/guides/prompts-advanced-usage.md#generated-knowledge-prompting) and try them here.

---